In [1]:
# Read in libraries
import os
import csv
import pandas as pd
from functools import reduce

# Set the directory path where your files are located
folder_path = "C:\\Users\\johnh\\OneDrive\\Documents\\GitHub\\water-supply-forecast\\assets\\data\\teleconnections"
folder_path_flow = "C:\\Users\\johnh\\OneDrive\\Documents\\GitHub\\water-supply-forecast\\assets\\data"
folder_path_grace = "C:\\Users\\johnh\\OneDrive\\Documents\\GitHub\\water-supply-forecast\\assets\\data\\grace_indicators"

In [2]:
# Dictionary to map month abbreviations to numeric values
month_to_num = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Dictionary to map month abbreviations to numeric values
month_to_num_up = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,
    'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8,
    'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

In [3]:
# Basic cleaning for mjo dataset
df_mjo = pd.read_table(os.path.join(folder_path,"mjo.txt"),delim_whitespace=True, skiprows=1)
df_mjo = df_mjo.iloc[1:]
df_mjo.columns = df_mjo.columns.str.strip()
df_mjo = df_mjo.add_prefix('mjo')
df_mjo = df_mjo[df_mjo['mjo20E'] != '*****'] # Remove future values (missing)

df_mjo['year'] = df_mjo['mjoPENTAD'].astype(str).str[:4].astype(int)
df_mjo['month'] = df_mjo['mjoPENTAD'].astype(str).str[4:6].astype(int)
df_mjo['day'] = df_mjo['mjoPENTAD'].astype(str).str[6:8].astype(int)

In [4]:
# Basic cleaning for nino dataset
df_nino = pd.read_table(os.path.join(folder_path,"nino_regions_sst.txt"),delim_whitespace=True)
df_nino = df_nino.rename(columns={'YR':'year', 'MON':'month'})
df_nino = df_nino.rename(columns={c: 'nino'+c for c in df_nino.columns if c not in ['year', 'month']})

In [6]:
# Basic cleaning for oni dataset
df_oni = pd.read_table(os.path.join(folder_path,"oni.txt"),delim_whitespace=True)
df_oni = df_oni.rename(columns={'YR':'year'})
df_oni['month'] = 1 #Assume month of collection is january
df_oni = df_oni.rename(columns={c: 'oni'+c for c in df_oni.columns if c not in ['year', 'month']})

In [7]:
# Basic cleaning for pdo dataset
df_pdo = pd.read_table(os.path.join(folder_path,"pdo.txt"),delim_whitespace=True,skiprows=1)
df_pdo = pd.melt(df_pdo, id_vars=['Year'], var_name='Month', value_name='pdo')
df_pdo = df_pdo.rename(columns={'Year':'year', 'Month':'month'})
df_pdo = df_pdo[df_pdo['pdo'] != 99.99] # Remove future values (missing)
df_pdo['month'] = df_pdo['month'].map(month_to_num)

In [8]:
# Basic cleaning for pna dataset
df_pna = pd.read_table(os.path.join(folder_path,"pna.txt"),delim_whitespace=True)
df_pna = pd.melt(df_pna, id_vars=['year'], var_name='month', value_name='pna')
df_pna['month'] = df_pna['month'].map(month_to_num)

In [9]:
# Basic cleaning for soi dataset
df_soi1 = pd.read_table(os.path.join(folder_path,"soi1.txt"),delim_whitespace=True,skiprows=3)
df_soi1.columns = df_soi1.columns.str.strip()
df_soi1 = pd.melt(df_soi1, id_vars=['YEAR'], var_name='month', value_name='soi_anom')
df_soi1 = df_soi1.rename(columns={'YEAR':'year'})
df_soi1['month'] = df_soi1['month'].map(month_to_num_up)

In [10]:
# Basic cleaning for soi dataset
df_soi2 = pd.read_table(os.path.join(folder_path,"soi2.txt"),delim_whitespace=True,skiprows=3)
df_soi2.columns = df_soi2.columns.str.strip()
df_soi2 = pd.melt(df_soi2, id_vars=['YEAR'], var_name='month', value_name='soi_sd')
df_soi2 = df_soi2.rename(columns={'YEAR':'year'})
df_soi2['month'] = df_soi2['month'].map(month_to_num_up)

In [40]:
# Merge code
data_frames = [df_mjo, df_nino, df_oni, df_pdo, df_pna, df_soi1, df_soi2]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['year', 'month'],
                                            how='outer'), data_frames)

# Re-order columns
column_order = ['year'] + ['month'] + [col for col in df_merged.columns if col != ['month','year']]
df_merged = df_merged[column_order]

In [20]:
# Read in streamflows
df_flow = pd.read_csv(os.path.join(folder_path_flow,"train_monthly_naturalized_flow.csv"))

In [21]:
# Merge code
data_frames = [df_mjo, df_nino, df_oni, df_pdo, df_pna, df_soi1, df_soi2, df_flow]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['year', 'month'],
                                            how='outer'), data_frames)

KeyError: 'day'

In [ ]:
# Basic cleaning for grace indicators
df_grace = pd.read_csv(os.path.join(folder_path_grace,"grace_aggregated.csv"))

# Convert 'time' to datetime format
df_grace['time'] = pd.to_datetime(df_grace['time'])

# Extract day, month, and year into separate columns
df_grace['day'] = df_grace['time'].dt.day
df_grace['month'] = df_grace['time'].dt.month
df_grace['year'] = df_grace['time'].dt.year

df_grace.drop('time', axis=1, inplace=True)

data_frames = [df_merged, ]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['year', 'month'],
                                            how='outer'), data_frames)

In [43]:
# Output the DataFrame to a CSV file
df_merged.to_csv('merged_dataframe.csv', index=False)  # Set index=False to exclude the index column

The following code is adjusts variable types, then standardises numeric variables and one-hot encodes categorical variables.

In [49]:
# Create a DataFrame to display variable names and data types
variable_types_df = pd.DataFrame({'Variable': df_merged.columns, 'Data Type': df_merged.dtypes.values})

# Iterate over columns
for column_name in df_merged.columns:
    # Check if the column name contains the substring 'mjo'
    if 'mjo' in column_name:
        # Convert values to float using pd.to_numeric
        df_merged[column_name] = pd.to_numeric(df_merged[column_name], errors='coerce')

In [45]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

def preprocess_column(df, column_name):
    # Skip preprocessing for specified columns
    if column_name in ["month", "year", "day"]:
        return df
    
    # Check the data type of the column
    column_dtype = df[column_name].dtype
    
    if column_dtype == 'object':
        # If it's a categorical variable, perform one-hot encoding
        df = pd.get_dummies(df, columns=[column_name], prefix=[column_name])
    elif column_dtype in ['int64', 'float64']:
        # If it's a numeric variable, standardize it
        scaler = StandardScaler()
        df[column_name] = scaler.fit_transform(df[[column_name]])
    elif column_dtype == 'bool':
        # If it's a binary variable, check if it's 0-1 coded and encode if not
        unique_values = df[column_name].unique()
        if set(unique_values) == {0, 1}:
            print(f"{column_name} is already 0-1 coded.")
        else:
            label_encoder = LabelEncoder()
            df[column_name] = label_encoder.fit_transform(df[column_name])
    else:
        print(f"Warning: Unsupported data type for column {column_name}")

    return df

def preprocess_dataframe(df):
    # Iterate over all columns in the DataFrame
    for column_name in df.columns:
        df = preprocess_column(df, column_name)

    return df

# Perform preprocessing on all columns
trans_vars = preprocess_dataframe(df_merged)

In [47]:
# Output the DataFrame to a CSV file
trans_vars.to_csv('transformed_vars.csv', index=False)  # Set index=False to exclude the index column